In [1]:
import cv2
import socket
import numpy as np
import pickle
import struct
import time

In [2]:
def start_server(host='0.0.0.0', port=12345):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(1)

    print("Server is running and waiting for a connection...")

    while True:
        try:
            conn, addr = server_socket.accept()
            print(f"Connected to {addr}")

            video_capture = cv2.VideoCapture(0)

            while True:
                ret, frame = video_capture.read()
                cv2.resize(frame, (640, 480))
                #cv2.resize(frame, (1280, 720))

                if not ret:
                    break

                # 压缩帧
                result, encoded_frame = cv2.imencode('.jpg', frame, [cv2.IMWRITE_JPEG_QUALITY, 70])
                
                # 发送数据长度然后发送图像数据
                data = pickle.dumps(encoded_frame)
                try:
                    conn.sendall(struct.pack("L", len(data)) + data)  # 先发送数据长度
                except (BrokenPipeError, ConnectionResetError):
                    print(f"Connection with {addr} lost. Waiting for a new connection...")
                    break
                    
                time.sleep(0.2)

        except Exception as e:
            print(f"An error occurred: {e}")
        
        finally:
            if 'video_capture' in locals():
                video_capture.release()
            if 'conn' in locals():
                conn.close()

In [ ]:
start_server('0.0.0.0',8188)

Server is running and waiting for a connection...
Connected to ('192.168.1.169', 47436)
Connection with ('192.168.1.169', 47436) lost. Waiting for a new connection...
Connected to ('192.168.1.169', 35402)
Connection with ('192.168.1.169', 35402) lost. Waiting for a new connection...
Connected to ('192.168.1.169', 52138)
